In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [3]:
df = pd.read_pickle('features/mfcc_features.pkl')
df.head()

,Feature,Class
0,"[-620.2583, 61.49415, -15.288396, 14.906147, -...",0.0
1,"[-563.3379, 18.238453, -2.3402789, 5.720763, -...",1.0
2,"[-736.4029, 55.818638, 9.05353, 1.8999938, -12...",0.0
3,"[-278.87955, 100.88817, -25.865995, 15.782074,...",1.0
4,"[-1131.3711, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0.0


In [4]:
X = np.array(df.Feature.to_list())
y = np.array(df.Class.to_list())

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

In [6]:
X_train.shape

(22040, 50)

In [7]:
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(50, 1)),
    LSTM(32),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [8]:
%%capture --no-stdout
import tensorflow_addons as tfa
# ADDRESS IMBALANCE:
from sklearn.utils.class_weight import compute_class_weight

In [9]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))
class_weight_dict

{0: 0.9513942847276181, 1: 1.0538395333269581}

In [10]:
tqdm_callback = tfa.callbacks.TQDMProgressBar()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, class_weight=class_weight_dict, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[tqdm_callback])

Training:   0%|                                         0/10 ETA: ?s,  ?epochs/s

Epoch 1/10


2024-10-27 19:17:09.137462: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


0/689                                                                 ETA: ?s - 

Epoch 1/10
689/689 [==============================] - 20s 26ms/step - loss: 0.5671 - accuracy: 0.7140 - val_loss: 0.5499 - val_accuracy: 0.7301
Epoch 2/10


...

Epoch 8/10
689/689 [==============================] - 18s 26ms/step - loss: 0.4516 - accuracy: 0.7855 - val_loss: 0.4487 - val_accuracy: 0.7862
Epoch 9/10


0/689                                                                 ETA: ?s - 

Epoch 9/10
689/689 [==============================] - 18s 26ms/step - loss: 0.4484 - accuracy: 0.7894 - val_loss: 0.4433 - val_accuracy: 0.7918
Epoch 10/10


0/689                                                                 ETA: ?s - 

Epoch 10/10
689/689 [==============================] - 18s 27ms/step - loss: 0.4425 - accuracy: 0.7914 - val_loss: 0.4342 - val_accuracy: 0.7947


In [11]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
test_accuracy

173/173 [==============================] - 1s 6ms/step - loss: 0.4342 - accuracy: 0.7947


0.7947368621826172

In [20]:
X_test

array([[-6.1661340e+02,  2.5801218e+01, -1.2701217e+01, ...,
         2.5360510e-01,  7.3844142e-02,  8.6273074e-02],
       [-4.1932272e+02,  3.9473335e+01, -1.0724197e+01, ...,
        -1.3737037e+00, -9.1674280e-01, -3.5803679e-01],
       [-3.2226410e+02,  6.2136887e+01, -4.4887539e+01, ...,
         9.6915597e-01,  5.4065347e-01,  1.3136777e+00],
       ...,
       [-5.6213153e+02,  3.3038864e+01, -9.6612329e+00, ...,
        -4.8124385e-01, -4.5902881e-01,  1.1036498e-01],
       [-5.3383826e+02,  9.6581793e+00, -4.4254532e+00, ...,
         2.0085005e-01,  2.7337646e-02,  2.7415276e-01],
       [-4.8069168e+02,  3.8707554e+01, -1.9147028e+01, ...,
         1.6952436e+00,  5.9119302e-01, -1.2798436e-01]], dtype=float32)

In [12]:
model.save('./saved_models/cnn_mfcc.keras')